In [1]:
lams = [[2**i, 2**j] for i in range(-6, 13, 2) for j in range(-6, 13, 2)] # permutation of all possible lams

In [2]:
from models_MR import *
from run_helpers import cross_val_acc
from datasets import *
import numpy as np

data, label, n_class = coil20()
print('running RVFL...')
acc = []
t = []
for lam in lams:
    _, model_accuracy, duration = cross_val_acc(data, label, n_class, RVFL, lam)
    acc.append(model_accuracy)
    t.append(duration)

max_index = np.argmax(acc, axis=0)[0]
opt_lam = lams[max_index]
print('Accuracy: ', acc[max_index][0], u"\u00B1", acc[max_index][1])
print('Lambda: ', opt_lam)
print('Train time: ', t[max_index][0])
print('Test time: ', t[max_index][1])

running RVFL...
[0.9069444444444444, 0.019194826334840626] [0.7311516523361206, 0.003704547882080078]
[0.9159722222222223, 0.02986111111111112] [0.7510350704193115, 0.005820512771606445]
[0.9229166666666666, 0.02312614695187044] [0.7467498064041138, 0.0033045291900634767]
[0.9145833333333334, 0.020844904194171735] [0.7531559944152832, 0.002892470359802246]
[0.9215277777777777, 0.01730546638107933] [0.7428818941116333, 0.0037347555160522463]
[0.9111111111111111, 0.01666666666666668] [0.7465346574783325, 0.004005622863769531]
[0.9027777777777777, 0.027950849718747377] [0.7621413230895996, 0.00576779842376709]
[0.8833333333333334, 0.020317692831010835] [0.770138168334961, 0.003820204734802246]
[0.8618055555555555, 0.025125072937674357] [0.7294543027877808, 0.0037101030349731444]
[0.8347222222222224, 0.029100453944106885] [0.7432546138763427, 0.004595661163330078]
[0.9291666666666666, 0.016077551253875316] [0.7354254722595215, 0.005085539817810058]
[0.93125, 0.020234447617129798] [0.750198

array([ 30.19287485, 606.44010263])